In [1]:
from qibo.core.circuit import Circuit
from qibo.gates import M, X, RX
import numpy as np
from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

from qililab.constants import DEFAULT_PLATFORM_NAME
from qililab.experiment import Experiment, settings
from qililab.utils import Loop

configuration = ConnectionConfiguration(
    user_id=3,
    username="qili-admin-test",
    api_key="d31d38f4-228e-4898-a0a4-4c4139d0f79f",
)

connection = API(configuration=configuration)

[Qibo 0.1.7|INFO|2022-05-30 16:48:03]: Using numpy backend on /CPU:0
[Qibo 0.1.7|WARNING|2022-05-30 16:48:03]: numpy backend uses `np.einsum` and supports CPU only. Consider installing the qibojit or qibotf backends for increased performance and to enable GPU acceleration.
[qibo-connection] 0.3.9|DEBUG|2022-05-30 16:48:04]: Qibo Quantum Service API SERVER URL: https://qilimanjaro.ddns.net:8080
[qibo-connection] 0.3.9|DEBUG|2022-05-30 16:48:05]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/authorisation-tokens
DEBUG:qiboconnection.config:Calling: https://qilimanjaro.ddns.net:8080/api/v1/authorisation-tokens
[qibo-connection] 0.3.9|DEBUG|2022-05-30 16:48:06]: Connection successfully established.
DEBUG:qiboconnection.config:Connection successfully established.
[qibo-connection] 0.3.9|INFO|2022-05-30 16:48:06]: Storing personal qibo configuration...
INFO:qiboconnection.config:Storing personal qibo configuration...


In [2]:
print(settings)

{
    "hardware_average": 1024,
    "software_average": 1,
    "repetition_duration": 200000,
    "translation": {
        "readout_pulse": {
            "amplitude": 0.4,
            "duration": 2000,
            "phase": 0
        },
        "delay_between_pulses": 0,
        "delay_before_readout": 50,
        "gate_duration": 100,
        "num_sigmas": 4,
        "drag_coefficient": 0
    }
}


In [3]:
settings.repetition_duration = 200000

In [4]:

circuit = Circuit(1)
circuit.add(X(0))
circuit.add(M(0))
experiment = Experiment(platform_name=DEFAULT_PLATFORM_NAME, sequences=circuit, settings=settings, experiment_name="Rabi")

[qililab] [0.3.0|INFO|2022-05-30 16:48:06]: Building platform
INFO:qililab.config:Building platform


In [5]:
%matplotlib widget
# %matplotlib notebook
experiment.set_parameter(category="signal_generator", id_=0, parameter="frequency", value=3.451759e9)
experiment.set_parameter(category="signal_generator", id_=1, parameter="frequency", value=7.347367e9)
loop1 = Loop(category="signal_generator", id_=1, parameter="frequency", start=7.34e9, stop=7.35e9, num=100)
loop2 = Loop(category="awg", id_=1, parameter="gain", start=0, stop=1, num=20)
results = experiment.execute(loops=[loop1, loop2], connection=connection)

[qililab] [0.3.0|INFO|2022-05-30 16:48:07]: Connecting to instrument qblox_qcm.
INFO:qililab.config:Connecting to instrument qblox_qcm.
[qililab] [0.3.0|INFO|2022-05-30 16:48:07]: Connecting to instrument rohde_schwarz.
INFO:qililab.config:Connecting to instrument rohde_schwarz.
[qililab] [0.3.0|INFO|2022-05-30 16:48:07]: Connecting to instrument qblox_qrm.
INFO:qililab.config:Connecting to instrument qblox_qrm.


Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/101901, firmware:4.2.76.0-4.30.046.295) in 0.13s


[qililab] [0.3.0|INFO|2022-05-30 16:48:08]: Connecting to instrument rohde_schwarz.
INFO:qililab.config:Connecting to instrument rohde_schwarz.
[qibo-connection] 0.3.9|DEBUG|2022-05-30 16:48:08]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/live-plotting
DEBUG:qiboconnection.config:Calling: https://qilimanjaro.ddns.net:8080/api/v1/live-plotting


Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/113049, firmware:4.2.76.0-4.30.046.295) in 0.12s


[qililab] [0.3.0|INFO|2022-05-30 16:48:09]: Closing instrument qblox_qcm.
INFO:qililab.config:Closing instrument qblox_qcm.
[qililab] [0.3.0|INFO|2022-05-30 16:48:09]: Closing instrument rohde_schwarz.
INFO:qililab.config:Closing instrument rohde_schwarz.
[qililab] [0.3.0|INFO|2022-05-30 16:48:09]: Closing instrument qblox_qrm.
INFO:qililab.config:Closing instrument qblox_qrm.
[qililab] [0.3.0|INFO|2022-05-30 16:48:09]: Closing instrument rohde_schwarz.
INFO:qililab.config:Closing instrument rohde_schwarz.
